In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pylab as plt
import scipy.stats
import matplotlib.ticker as ticker
from tqdm import tqdm
import warnings
# warnings.filterwarnings("ignore")
from trade import BackTest
os.chdir("../") 

In [8]:
strategyDir = 'strategy_1226'
resDir = 'res_1226'

In [11]:
CAPITAL = 10**7
DAY  = 242
hedgeList = [0,0.1,0.2,0.3,0.4]
index =pd.read_csv('data/index.csv')
hs300 = index.loc[index['code'] == 300,['date','close']].set_index('date')
start = '2010-12-31'
end = '2022-06-30'
tmpI = hs300.loc[start:end]
tmpI = tmpI/tmpI.iloc[0][0]
tmpI.ffill(inplace=True)

rf = pd.read_csv('data/risk_free.csv')
rf.set_index('Clsdt',inplace = True)
dailyRf = rf.loc['2011-01-01':'2022-06-30','Nrrdaydt']
anualizedRF = (((dailyRf/100+1).prod())**(1/11.5)-1)*100

In [12]:
def max_drawdown(returns):
    i = np.argmax((np.maximum.accumulate(returns) - returns) / np.maximum.accumulate(returns))  # 结束位置
    if i == 0:
        return 0
    j = np.argmax(returns[:i])  # 开始位置
    return (returns[j] - returns[i]) / (returns[j])
def get_stats(asset,fee = True):
    tmpStats = {}
    nDay = asset.shape[0]
    tmpStats['Annualized Return (%)']=((asset.iloc[-1,0]/asset.iloc[0,0])**(DAY/nDay)-1)* 100
    tmpStats['Annualized Std (%)']=asset['dRet'].std() * np.sqrt(DAY) * 100
    tmpStats['Sharpe'] = (tmpStats['Annualized Return (%)']-anualizedRF)/tmpStats['Annualized Std (%)']
    tmpStats['Max Drawdown (%)'] = max_drawdown(asset['total_asset']) * 100
    if fee:
        tmpStats['Fees'] = asset['long_fee'].sum()+asset['short_fee'].sum()
    tmpStats = pd.DataFrame([tmpStats]).T
    return tmpStats

In [13]:
def analyze(name,h,show = False):
    # read data
    path = f'{resDir}/{name}/{name}_{h}/'
    bt = pd.read_csv(f'{path}asset.csv').set_index('date')
    bt.describe().round(2).to_csv(f'{path}stats.csv')

    # 画空头多头持仓
    plt.figure(figsize = (20,10),dpi = 300,facecolor = 'white')
    ax1 = plt.subplot(211)
    plt.plot(bt['cash']+bt['long_holding'],c = 'cadetblue')
    plt.plot(bt['total_asset'],c = 'indianred')
    plt.legend(['Cash+Stock Value','Real Asset'])
    ax1.xaxis.set_major_locator(ticker.MultipleLocator(200))
    plt.title('Error Brought by Short')


    ax2 = plt.subplot(212)
    plt.plot(bt['long_holding'],c = 'cadetblue')
    plt.plot(bt['short_holding'],c = 'indianred')
    plt.legend(['Stock Value','Future Value'])
    ax2.xaxis.set_major_locator(ticker.MultipleLocator(200))
    plt.title('Long VS Short')
    plt.suptitle(f'{name} Hedge: {h*100}%')
    plt.savefig(f'{path}long_short.png', bbox_inches='tight')
    if show:
        plt.show()
    plt.close()


    # prepare data
    bt.loc['2010-12-31','total_asset'] = CAPITAL
    bt.sort_index(inplace = True)
    bt['pnl'] = bt['total_asset'].diff()
    bt['total_asset'] = bt['total_asset']/CAPITAL
    bt['year']=bt.index.str.slice(0,4)
    bt['dRet']=bt['total_asset'].diff()/bt['total_asset'].shift(1)

    # get stats
    stats = bt.groupby('year').apply(lambda x:get_stats(x))[0].unstack()
    allT = get_stats(bt).T
    allT.index = ['AllTime']
    stats = stats.append(allT).round(2)
    stats = stats.iloc[1:]
    stats.to_csv(f'{path}performance.csv')
    if show:
        print(stats)

    # plot 净值曲线
    start = '2010-12-31'
    end = '2022-06-30'
    plt.figure(figsize = (20,10),dpi = 300,facecolor = 'white')
    ax1 = plt.subplot(211)
    plt.grid(axis = 'y',c = 'gainsboro')
    # plot asset
    tmpAsset = bt.loc[start:end,'total_asset']
    tmpAsset = tmpAsset/tmpAsset[0]
    plt.plot(tmpAsset,c = '#b23a48') 
    # plot hs300
    tmpI = hs300.loc[start:end]
    tmpI = tmpI/tmpI.iloc[0][0]
    tmpI.ffill(inplace=True)
    plt.plot(tmpI,c = '#86bbd8')
    # set plotting
    plt.legend(['cum return','hs300'])
    ax1.xaxis.set_major_locator(ticker.MultipleLocator(200))
    ax1.set_title(f'Cumulative Returns')
    # 画pnl
    ax2 = plt.subplot(212)
    plt.grid(axis = 'y',c = 'gainsboro')
    plt.axhline(0, color='grey', linestyle='--')
    ax2.plot(bt.index,bt['pnl'],linestyle=':',c = 'cadetblue')
    ax2.scatter(bt.index,bt['pnl'],c=bt['pnl'], cmap="coolwarm")
    ax2.xaxis.set_major_locator(ticker.MultipleLocator(200))
    ax2.set_title(f'PNL')

    plt.suptitle(f'{name} Hedge: {h*100}%')
    plt.savefig(f'{path}asset.png', bbox_inches='tight')
    if show:
        plt.show()
    plt.close()


    # calculate holdings
    holding = pd.read_csv(f'{path}stock_records.csv').set_index(['date','stock'])
    numOfStk = holding.groupby('date').count().iloc[:,0]
    numOfStk.name = 'numOfStk'
    bt = pd.merge(bt,numOfStk,right_index=True,left_index = True,how = 'outer')
    bt['numOfStk'] = bt['numOfStk'].fillna(0)

    # plot holdings
    plt.figure(figsize = (20,10),dpi = 300,facecolor = 'white')
    ax1 = plt.subplot(211)
    plt.grid(axis = 'y',c = 'gainsboro')
    plt.scatter(bt.index,bt['numOfStk'],color = '#86bbd8')

    # set plotting
    ax1.xaxis.set_major_locator(ticker.MultipleLocator(200))
    ax1.set_title(f'Position')

    # plot long_fee
    ax2 = plt.subplot(212)
    plt.grid(axis = 'y',c = 'gainsboro')

    longF = bt['long_fee'].copy()
    longF[longF==0] = np.nan
    shortF = bt['short_fee'].copy()
    shortF[shortF==0] = np.nan
    plt.scatter(bt.index,longF,color = '#86bbd8')
    plt.scatter(bt.index,shortF,color = 'indianred')

    # set plotting
    ax2.xaxis.set_major_locator(ticker.MultipleLocator(200))
    ax2.set_title(f'Fee')
    plt.suptitle(f'{name} Hedge: {h*100}%')
    plt.savefig(f'{path}position.png', bbox_inches='tight')
    if show:
        plt.show()
    plt.close()


    # target position
    target = pd.read_excel(f'{strategyDir}/strategy_{name}.xlsx')
    target['month'] = target['trade_date'].str.slice(0,7)
    target.set_index('month',inplace=True)
    target.drop('trade_date',axis=1,inplace = True)
    target[target!=0] = 1
    target = target.sum(axis=1)
    target.loc['2022-06']=0
    target = target.shift(1)
    target.name = 'target'

    # real position
    numOfStk.index = numOfStk.index.str.slice(0,7)
    numOfStk.drop_duplicates(inplace=True)
    numOfStk.name = 'real'

    # compare position
    comparePos = pd.merge(target,numOfStk,left_index=True,right_index=True)
    comparePos['diff'] = comparePos['target'] - comparePos['real']
    compareRes = comparePos.describe()
    compareRes.round(2).to_csv(f'{path}compare_position.csv')

In [14]:
def analyze_all_hedge(name,show=False):
    colNames = [f'{int(i*100)}%' for i in hedgeList]
    allHedge = pd.DataFrame()
    allPerformance = pd.DataFrame()
    for h in hedgeList:
        path = f'{resDir}/{name}/{name}_{h}/'
        # asset
        bt = pd.read_csv(f'{path}asset.csv').set_index('date')
        allHedge=pd.concat([allHedge,bt['total_asset']],axis=1)
        # performance
        performance = pd.read_csv(f'{path}performance.csv').set_index('Unnamed: 0')
        allPerformance=pd.concat([allPerformance,performance.loc['AllTime']],axis=1)

    allPerformance.columns = colNames
    allPerformance.index.name = 'Hedge Ratio'
    allPerformance.to_csv(f'{resDir}/{name}/performance_all_hedge.csv')

    allHedge=pd.concat([allHedge/CAPITAL,tmpI],axis=1)
    allHedge.loc['2010-12-31'] = 1
    allHedge.sort_index(inplace=True)
    allHedge.columns =colNames+['HS300']
    col = ['#a1cca5', '#8fb996', '#709775', '#415d43', '#111d13']
    plt.figure(facecolor = 'white',figsize=(20,5),dpi=300)
    ax1 = plt.subplot(111)
    ax1.xaxis.set_major_locator(ticker.MultipleLocator(200))
    for i in range(len(allHedge.columns)):
        if i == len(allHedge.columns)-1:
            plt.plot(allHedge.iloc[:,i],c='#f7a399')
        else:
            plt.plot(allHedge.iloc[:,i],c=col[i])
    plt.legend(allHedge.columns)
    plt.title(f'{name} with different hedge ratios')
    plt.savefig(f'{resDir}/{name}/asset_all_hedge.png', bbox_inches='tight')
    if show:
        plt.show()
    plt.close()

In [15]:
# ## index
# names = {300:'HS300',905:'ZZ500',906:'ZZ800',852:'ZZ1000'}
# number = {300:1,905:2,906:3,852:4}
# def get_index_stats(code):
#     bt = index.loc[index['code'] == code,['date','close']].set_index('date')
#     bt.columns = ['total_asset']
#     bt['pnl'] = bt['total_asset'].diff()
#     bt['total_asset'] = bt['total_asset']/CAPITAL
#     bt['year']=bt.index.str.slice(0,4)
#     bt['dRet']=bt['total_asset'].diff()/bt['total_asset'].shift(1)
#     # get stats
#     stats = bt.groupby('year').apply(lambda x:get_stats(x,False))[0].unstack()
#     allT = get_stats(bt,False).T
#     allT.index = ['AllTime']
#     stats = stats.append(allT).round(2)
#     stats = stats.iloc[1:]
#     stats['code'] = names[code]
#     stats['number'] = number[code]
#     return stats
# indices = pd.DataFrame()
# for code in [300,905,906,852]:
#     indices = indices.append(get_index_stats(code))
# indices = indices.set_index([indices.index,'number','code',]).sort_index().droplevel(1)
# indices.to_csv('res/index_stats.csv')

In [16]:
for sty in os.listdir(strategyDir):
    if sty.startswith('strategy'):
        name = sty[9:-5]
        print(name)
        for hedge in hedgeList:
            analyze(name,hedge)
        analyze_all_hedge(name)

dacc_ew


KeyError: "None of ['date'] are in the columns"

In [9]:
# name = 'monthly_roa_q_ew'
# for hedge in hedgeList:
#     analyze(name,hedge)
# analyze_all_hedge(name)

# ('res_prof/quarterly_roa_ttm_vw/quarterly_roa_ttm_vw_0.4',
#  'strategy_profitability/strategy_quarterly_roa_ttm_vw.xlsx',
#  0.4)
name = 'monthly_roa_ttm_vw'
hedge = 0.4

In [10]:
# analyze(name,hedge)